In [84]:
import os
import datetime
import pandas as pd
import numpy as np
import warnings

Define filepaths. You will need to change the filepaths to reflect your own directory structure. Just copy and paste into the "".
For reading in the HRV files, the below list comprehension looks for any files starting with 'p' and ending with '.csv'.
This reflects the file naming convention used for the HRV files. If you rename files or the naming convention is changed, then you will have to adapt this to reflect the changes.

In [85]:
main_dir = r"P:\Spironolactone\main_qualtrics"
hrv_dir = r"P:\Spironolactone\Firstbeat"
hrv_files = [file for file in os.listdir(hrv_dir) if file.lower().startswith('p') and file.endswith('.csv')]

Make an output directory (or issue a warning if it already exists). I put mine into the directory where the rest of the HRV files sit and named it "processed_hrv_files" but you are of course free to choose your own name and location.

In [86]:
output_dir = os.path.join(hrv_dir,"processed_hrv_files")
try:
    os.makedirs(output_dir)
except OSError:
    # if directory already exists
    warnings.warn("Directory already exists. Files may be overwritten. Manual check advised.")
    pass



<ipython-input-86-3de6f22e24f9>:6: UserWarning: Directory already exists. Files may be overwritten. Manual check advised.
  warnings.warn("Directory already exists. Files may be overwritten. Manual check advised.")


Reading in the qualtrics file from the study day session and retaining/renaming the columns we need below.
I would NOT advise changing the 'new_names' unless you are happy to change them in the code below.

In [87]:
col_list =  ["Status","DQ-1","Firstbeat_on_time","baseline start","baseline end","Q645","Q646","FILM-START","Q648","Q649"]
new_names = ["response_type","Participant_number","Firstbeat_start","RT1_start","RT1_end","RT2_start","RT2_end","Film_start","RT3_start","RT3_end"]
qualtrics_df = pd.read_csv(os.path.join(main_dir,"main_dat.csv"),usecols =col_list,skiprows= [1,2])
qualtrics_df.columns = new_names

Utilities.
All the functions used in the main body of the script below are defined here.

In [88]:
def remove_invalid_records(in_df, exclude_pnums = None):
    """
    remove invalid records - participant id is nan or
    >100 (usually indicates test record)

    Parameters
    ----------
    in_df:  pd DataFrame
        dataframe to operate on
    exclude_pnums: list[int]
        specify participants who should be 
        excluded from the analysis, if any

    Returns
    -------
        dataframe w/o the above records
    """
    in_df = in_df[(in_df.Participant_number.notna())
                    &(in_df.Participant_number<100)]
    if exclude_pnums:
        in_df = in_df.drop(labels = in_df[
                in_df.Participant_number.isin(exclude_pnums)].index,
                axis = 0)
    return in_df
    
def remove_duplicate_participants(in_df):
    """
    If we have duplicate records
    for a given participant, remove
    that with the most NaNs.

    Parameters
    ----------
    in_df:  pd Dataframe

    Returns
    -------
        in_df w/o all nan duplicates
    """
    duplicated = in_df.loc[in_df.duplicated(subset = "Participant_number"),"Participant_number"]
    if duplicated is None:
        return in_df
    else:
        drop_inds = []
        for dup in duplicated:
            nan_sum = in_df[in_df.Participant_number == dup].isna().sum(axis = 1)
            nan_max_ind = nan_sum[nan_sum == nan_sum.max()].index
            drop_inds.append(nan_max_ind[0])
        in_df = in_df.drop(labels = drop_inds, axis = 0)
        return in_df       

def flag_duplicate_participants(in_df):
    """
    If we have duplicate records
    for a given participant, flag
    them.

    Parameters
    ----------
    in_df:  pd Dataframe

    Returns
    -------
        participant numbers for those
        with duplicate records
    """
    duplicated = in_df.loc[in_df.duplicated(subset = "Participant_number"),"Participant_number"]
    print(f"The following participants have duplicate records:\n{duplicated.values}")
    return duplicated         

def convert_time_cols(in_df):
    """
    convert time cols to datetime format
    
    Parameters
    ----------
    in_df:  pd Dataframe
        dataframe to operate on
    
    Returns
    -------
    dataframe with time cols converted
    to datetime format.
    """
    time_cols = [col for col in in_df.columns
                if any(k in col for k in ["start","end"])]
    in_df.loc[:,time_cols] = in_df.loc[:,time_cols].apply(
                                lambda x: pd.to_datetime(x,errors = "coerce"),
                                axis = 1)
    return in_df

def add_end_time(in_df,start_time_col, amount):
    """
    If we don't have a time for the interval end,
    provide an end time in minutes from start time.

    Parameters
    ----------
    in_df:  pd dataframe
        dataframe to operate on
    start_time_col: str
        name of column with interval start time
        eg. Film_start
    amount: int
        number of minutes to add to start time
    
    Returns
    -------
    in_df with end_time column added.
    """
    new_col_name = "_".join([start_time_col.split("_")[0],"end"])
    in_df[new_col_name] = in_df[start_time_col] + datetime.timedelta(minutes = amount)
    return in_df

def make_rel_time_cols(in_df,time_col_start,time_col_end):
    """
    calculate relative timings for time columns.

    Parameters
    ----------
    in_df:  pandas Dataframe
        dataframe to operate on
    time_col_start: str
        name of start time column
    time_col_end:   str
        name of end time column
    
    Returns
    -------
    in_df with rel time columns added
    """
    new_col_name = "_".join([time_col_end,"interval"])
    in_df[new_col_name] = in_df[time_col_end]-in_df[time_col_start]
    return in_df

def convert_to_secs(in_df):
    """
    convert time delta to seconds.

    Parameters
    ----------
    in_df:  pd Dataframe
        dataframe to operate on
    
    Returns
    -------
    interval cols converted to secs
    from Firstbeat start time.
    """
    interval_cols = in_df.filter(like = "interval",axis = 1).columns
    in_df.loc[:,interval_cols] = in_df.loc[:,interval_cols].applymap(
                                            lambda x: x.total_seconds())
    return in_df

def select_hrv_record(pnum,hrv_files):
    """
    select hrv file for participant
    
    Parameters
    ----------
    pnum:   int or float
        participant number whos record
        needs to be retrieved
    hrv_files:  list[str]
        list of hrv files
    
    Returns
    -------
        name of hrv file for participant pnum (str)
    """
    recs = [file for file in hrv_files if int(file[1:4])==pnum]
    if len(recs)>1:
        warnings.warn(f"Found more than one file for participant {pnum}.\nManual check advised.")
    return recs.pop()

def get_time_stamp(in_df,interval_col,pnum):
    """
    Get time stamp for processing HRV files.

    Parameters
    ----------
    in_df:  pd Dataframe
        input dataframe
    interval_col:   str
        name of column with desired interval
        eg "Film_start_interval"
    pnum:   int
        participant number
    
    Returns
    -------
    time stamp (for input to get_hrv_interval())
    """
    time_stamp = in_df[in_df.Participant_number==pnum].reset_index().at[0,interval_col]
    return time_stamp

def get_hrv_interval(hrv_df,interval_start:float,interval_end:float):
    """
    Get intervals for HRV data
    This function will find the closest value to the
    specified start and end times.

    Parameters
    ----------
    hrv_df: pd DataFrame
        HRV data for a given participant
    interval_start: float
        start time as duration in seconds from
        start of Firstbeat (ie Film_start for participant 1)
    interval_end:   float

    """
    hrv_df["IBI_cumsum"] = hrv_df.IB_intervals.cumsum()/1000
    start_vals = (hrv_df.IBI_cumsum-interval_start).sub(0).abs().idxmin()
    end_vals = (hrv_df.IBI_cumsum-interval_end).sub(0).abs().idxmin()
    interval_df = hrv_df.iloc[start_vals:end_vals]
    return interval_df["IB_intervals"]

Preprocessing.
Remove_invalid_records: Gets rid of records with invalid or missing participant numbers. Invalid is defined as >100. This usually means that we were testing the qualtrics survey (we often used participant numbers such as 999).
flag_duplicate_participants: Print participant numbers for participants who have more than one record in the main qualtrics survey.
remove_duplicate_participants: remove duplicate records based on which of the two has more NaNs. You can comment out this step if you would not like to remove them, but this will leave you with two records and will most likely cause errors below.
If you would like to remove one record manually, I would advise retrieving the duplicate records like so: qualtrics_df[qualtrics_df.Participant_number == pnum] where pnum is the participant number that was flagged as having duplicates. This will show you all records and you could then remove one of them (eg using qualtrics_df.drop(labels = pnum_index, axis = 0), where pnum_index is the row index of the record your want to drop).
convert_time_cols: convert time related columns to datetime format
add_end_time: Just used for the film. This is because there was no automatic time capture to indicate the end of the film. Basically, I just added 15 minutes to the film start time to get this. You can adjust this by changing the second positional argument to whatever number of minutes you think is best (it has to be in minutes).
make_rel_time_cols: Calculate the interval from the start of the firstbeat recording to the different interval start/end times (Film, RTs)
convert_to_secs: Convert the above interval columns to seconds.

In [89]:
qualtrics_df = remove_invalid_records(qualtrics_df,exclude_pnums = [1])
duplicates = flag_duplicate_participants(qualtrics_df)
qualtrics_df = remove_duplicate_participants(qualtrics_df)
qualtrics_df = convert_time_cols(qualtrics_df)
qualtrics_df = add_end_time(qualtrics_df,"Film_start",15)
rt_time_cols = [f for f in qualtrics_df.columns if any(k in f for k in ["start","end"])]
for rt_time in rt_time_cols[1:]:
    qualtrics_df = make_rel_time_cols(qualtrics_df,rt_time_cols[0],rt_time)
qualtrics_df = convert_to_secs(qualtrics_df)


The following participants have duplicate records:
[4.]


This is where we process the actual HRV files. I've commented the main steps below, but here's the gist:
For each participant, load the hrv file, get the start/end intervals for each of the different sections (Film, RTs)
get_hrv_intervals gets the relevant sections from a participant's HRV file by calculating the cumulative sum of the interbeat intervals, subtracting interval start/end times from this and identifying the index of the row where the result of this subtraction is closes to 0.
The resulting files, containing HRV data for each individual and participant, are then saved to output_dir.
If there is something wrong with the HRV file or the interval data (for instance, because a tag was missed or the Firstbeat cut out, resulting in missing data), this will result in a warning and the participant number and interval named being appended to the missing_pnums list. You can then access this following processing of the hrv files to conduct manual checks on those participants to see what happened. Mostly it seems to be the firstbeat having cut out, resulting in no data for some intervals. Note that NO csv file will be saved for affected participants/intervals.

In [90]:
# Zip up start/end interval column names for easier access in for loop below:
start_intervals = qualtrics_df.filter(like = "start_interval", axis = 1).columns.sort_values()
end_intervals = qualtrics_df.filter(like = "end_interval", axis = 1).columns.sort_values()
intervals = list(zip(start_intervals, end_intervals))

# Track participants whose HRV data for any of the intervals is missing
missing_pnums = []
for pnum in qualtrics_df.Participant_number:
    # check if file exists
    try:
        my_rec = select_hrv_record(pnum,hrv_files)
    except IndexError:
        print(f"No HRV file found for participant {pnum}.")
        continue
    # read in HRV file for participant pnum
    hrv_df = pd.read_csv(
                        os.path.join(hrv_dir,my_rec),
                        header = 0, names = ["IB_intervals"],
                        skiprows = np.arange(0,4)
                        )
    # select the part of the HRV file that corresponds to given interval
    # do this for all intervals (Film, RT1, RT2, RT3)
    for start_interval, end_interval in intervals:
        start_time = get_time_stamp(qualtrics_df, start_interval, pnum)
        end_time = get_time_stamp(qualtrics_df, end_interval, pnum)
        interval_df = get_hrv_interval(hrv_df,start_time,end_time)
        # if the resulting dataframe is empty, flag this and hold on to pnum/interval
        if interval_df.empty:
            interval_name = start_interval.split("_")[0]
            warnings.warn(f"{pnum} has no valid data for {interval_name} interval.\nManual check advised.")
            missing_pnums.append([pnum,interval_name])
            continue
        # save to file
        interval_df.to_csv(os.path.join(output_dir, "_".join([start_interval.split("_")[0],str(pnum),"hrv.csv"])),index = False)

<ipython-input-90-e65adcc6b827>:30: UserWarning: 7.0 has no valid data for Film interval.
Manual check advised.
  warnings.warn(f"{pnum} has no valid data for {interval_name} interval.\nManual check advised.")
<ipython-input-90-e65adcc6b827>:30: UserWarning: 7.0 has no valid data for RT1 interval.
Manual check advised.
  warnings.warn(f"{pnum} has no valid data for {interval_name} interval.\nManual check advised.")
<ipython-input-90-e65adcc6b827>:30: UserWarning: 7.0 has no valid data for RT2 interval.
Manual check advised.
  warnings.warn(f"{pnum} has no valid data for {interval_name} interval.\nManual check advised.")
<ipython-input-90-e65adcc6b827>:30: UserWarning: 7.0 has no valid data for RT3 interval.
Manual check advised.
  warnings.warn(f"{pnum} has no valid data for {interval_name} interval.\nManual check advised.")
<ipython-input-90-e65adcc6b827>:30: UserWarning: 10.0 has no valid data for Film interval.
Manual check advised.
  warnings.warn(f"{pnum} has no valid data for {in

No HRV file found for participant 11.0.


<ipython-input-88-d06b9b36b699>:173: UserWarning: Found more than one file for participant 25.0.
Manual check advised.
  warnings.warn(f"Found more than one file for participant {pnum}.\nManual check advised.")
<ipython-input-90-e65adcc6b827>:30: UserWarning: 25.0 has no valid data for Film interval.
Manual check advised.
  warnings.warn(f"{pnum} has no valid data for {interval_name} interval.\nManual check advised.")
<ipython-input-90-e65adcc6b827>:30: UserWarning: 25.0 has no valid data for RT2 interval.
Manual check advised.
  warnings.warn(f"{pnum} has no valid data for {interval_name} interval.\nManual check advised.")
<ipython-input-90-e65adcc6b827>:30: UserWarning: 25.0 has no valid data for RT3 interval.
Manual check advised.
  warnings.warn(f"{pnum} has no valid data for {interval_name} interval.\nManual check advised.")
<ipython-input-90-e65adcc6b827>:30: UserWarning: 13.0 has no valid data for RT2 interval.
Manual check advised.
  warnings.warn(f"{pnum} has no valid data for

No HRV file found for participant 37.0.
No HRV file found for participant 38.0.
No HRV file found for participant 39.0.
No HRV file found for participant 40.0.
No HRV file found for participant 41.0.


<ipython-input-88-d06b9b36b699>:173: UserWarning: Found more than one file for participant 34.0.
Manual check advised.
  warnings.warn(f"Found more than one file for participant {pnum}.\nManual check advised.")
